In [99]:
import pandas as pd
import os
import gzip
import pickle
import openai
import torch.nn.functional as F
import torch

In [100]:
def get_openai_response(prompt, tokens = 2000, model="gpt-3.5-turbo-instruct"):
    if model.__contains__("instruct"):
        response = openai.Completion.create(
            model=model,
            prompt=prompt,
            temperature=0,
            max_tokens=tokens,
            top_p=1,
            logit_bias = {
                "198": -100, # new lines
                },
            logprobs= 15,
        )
        text = response.choices[0].text

        return text
    else:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant"
                },
                {
                    "role": "user",
                    "content": prompt
                }
                ],
            temperature=0,
            max_tokens=tokens,
            top_p=1,
        )
        text = response.choices[0].message.content


        # print("returning ai text:", text)
        return text

# hardware name

In [101]:
# categories = [
#     "groupbuys",
#     "hardware",
#     "miners",
#     "mining",
#     "mining_support",
#     # "pools",
# ]

# df = pd.DataFrame()

# #load every csv in the folder and append them
# for cat in categories:
#     with gzip.open('cleaned-data/'+cat+'.pkl.gz', 'rb') as f:
#         df_cat = pickle.load(f)
#         df_cat['category'] = cat
#         df = pd.concat([df, df_cat], ignore_index=True)

# path = "datasets/extracted/"
# file_name = "hardware_name.csv"

# prompt_start = """

# """.strip()

# prompt_end = """
# Analyze if the thread contains a name of mining hardware. Write either "Hardware found: Nothing" or "Hardware found: <hardware name>".



# Hardware found:
# """.strip()


# if not os.path.exists(path+file_name):
#     dataset = pd.DataFrame()

#     thread_count = 0
#     for (id,row) in df.sample(10000).iterrows():
#         if(len(row["post"]) < 100):
#             continue

#         date = row["dates"].split("<sep>")[-1]

#         thread = ""
#         # thread += "Category: " + row["category"] + "\n"
#         thread += "Date: " + date[:7] + "\n"
#         thread += "Topic: " + row["topic"] + "\n"        
#         thread += "### Original post:\n"
#         i = 1
#         for (post, date) in zip(row["post"].split("<sep>"), row["dates"].split("<sep>")):
#             if len(post) > 800:
#                 thread += post[:800] + "<rest of post truncated>\n\n"
#                 thread += f"### Reply {i}:\n"
#                 i += 1
#             elif len(post) < 5:
#                 pass
#             else:
#                 thread += post + "\n\n"
#                 thread += f"### Reply {i}:\n"
#                 i += 1
#         #remove the last line
#         thread = thread[:-len(f"### Reply {i-1}:\n")]
#         if len(thread) > 4000:
#             thread = thread[:4000] + "<rest of thread truncated>\n"
        
#         print(thread)

#         actual_prompt = prompt_start + "" + thread + "\n\n\n\n" + prompt_end + ""
#         # print(actual_prompt)

#         response = get_openai_response(actual_prompt, tokens = 20)
#         print("response:", response, "\n\n\n\n\n\n")

#         if "Nothing" in response:
#             continue

#         dataset = pd.concat([dataset, pd.DataFrame([{
#             "date": date[:-9],
#             "hardware_name": response.replace("Hardware found: ", ""),
#             }])], ignore_index=True)

#         thread_count += 1  
#         if thread_count > 5:
#             break
        
#     print(dataset.value_counts())

#     #save the dataset
#     dataset.to_csv(path+file_name, index=True)

# hardware price

In [102]:
categories = [
    "groupbuys",
    "hardware",
    "miners",
    "mining",
    "mining_support",
    # "pools",
]

df = pd.DataFrame()

#load every csv in the folder and append them
for cat in categories:
    with gzip.open('cleaned-data/'+cat+'.pkl.gz', 'rb') as f:
        df_cat = pickle.load(f)
        df_cat['category'] = cat
        df = pd.concat([df, df_cat], ignore_index=True)

path = "datasets/extracted/"
file_name = "name_and_price.csv"

prompt_start = """

""".strip()

prompt_end = """
Analyze if this thread contains a name and price of mining hardware.
If the thread has no hardware name, write:
"Hardware found: Nothing"
If the thread only has a name with no price, write:
"Hardware found: <hardware name> for unknown price"
If the thread has both a hardware name and a corresponding price, write:
"Hardware found: <hardware name> for <hardware price>"
If the thread has multiple mentions of mining hardware, write all of them down, separated with a comma.



Hardware found:
""".strip()


if not os.path.exists(path+file_name):
    dataset = pd.DataFrame()
else:
    dataset = pd.read_csv(path+file_name)

thread_count = 0

for (id,row) in df.sample(15476).iterrows():
    if(len(row["post"]) < 100):
        continue

    date = row["dates"].split("<sep>")[-1]

    thread = ""
    # thread += "Category: " + row["category"] + "\n"
    thread += "Date: " + date[:7] + "\n"
    thread += "Topic: " + row["topic"] + "\n"        
    thread += "### Original post:\n"
    i = 1
    for (post, date) in zip(row["post"].split("<sep>"), row["dates"].split("<sep>")):
        if len(post) > 800:
            thread += post[:800] + "<rest of post truncated>\n\n"
            thread += f"### Reply {i}:\n"
            i += 1
        elif len(post) < 5:
            pass
        else:
            thread += post + "\n\n"
            thread += f"### Reply {i}:\n"
            i += 1
    #remove the last line
    thread = thread[:-len(f"### Reply {i-1}:\n")]
    if len(thread) > 4000:
        thread = thread[:4000] + "<rest of thread truncated>\n"

    whitelist = [
        "$",
        "usd",
        "dollar",
        "€",
        "eur",
        "euro",
        "£",
        "gbp",
        "pound",
        "yen",
        "jpy",
        "cny",
        "rmb",
        "yuan",
        "ruble",
        "btc",
        "bitcoin",
    ]

    if not any([x in thread.lower() for x in whitelist]):
        continue
    
    print(thread)

    actual_prompt = prompt_start + "" + thread + "\n\n\n\n" + prompt_end + ""
    # print(actual_prompt)

    response = get_openai_response(actual_prompt, tokens = 100).strip().rstrip(',')

    print("response:", response, "\n\n\n\n\n\n")

    if "Nothing" in response:
        continue

    response = response.replace("Hardware found: ", "").replace("<hardware price>", "").replace("<hardware name>", "").strip()

    responses = response.split(", ")

    for r in responses:
        try:
            r2 = r.split(" for ")
            name = r2[0]
            price = r2[1]
        except:
            name = r
            price = "unknown price"
        dataset = pd.concat([dataset, pd.DataFrame([{
            "date": date[:-9],
            "hardware_name": name,
            "hardware_price": price,
            }])], ignore_index=True)


    thread_count += 1  
    if thread_count > 700:
        break
    
print(dataset.value_counts())

dataset = dataset.sort_values(by=['date'])

#save the dataset
dataset.to_csv(path+file_name, index=False)

Date: 2013-06
Topic: MineMeBitcoins v1.2.1 - GPU and CPU mining, made simple.
### Original post:
Version 1.3 is up and changes the timing of the internet check, fixes a bug which can occur when downloading the necessary files.


response: Nothing 






Date: 2013-11
Topic: R14X Payment Details
### Original post:
Round 14x Payment Details (Similar to R7-R11, for the most <truncated> Orders should be placed here: <link> Payments will be directed to thomas_s via our site for R14x but I will have co-admin to this wallet for auditing or for payments if we're in a rush. We've worked together on 12 different Group Buys with great success including 2 of the fastest GB sellouts in the forum's history for Industrial class Rigs.For US Customers: for a 3% fee for USD conversion to BTC, we recommend: <truncated> a FinCen compliant MSB who can get BTC sent to the wallet address you want in as little as 15 minutes to an hour.For Intl. Customers: We are still exploring suggestions for inexpensive BTC

In [103]:
dataset = dataset.sort_values(by=['date'])

#save the dataset
dataset.to_csv(path+file_name, index=False)